#  📌 비슷한 애니메이션 추천(TF-IDF)
### ✏️ DB 연결

In [186]:
import dotenv
import os
from pymongo import MongoClient

# 환경변수 불러오기
dotenv.load_dotenv(dotenv.find_dotenv())
USER = os.environ["MONGODB_USER"] # MongoDB user
PASSWORD = os.environ["MONGODB_PW"] # MongoDB password
PORT = int(os.environ["MONGODB_PORT"]) # MongoDB port

# DB 연결
client = MongoClient("mongodb://" + USER + ":" + PASSWORD + "@j7e104.p.ssafy.io", PORT)

db = client.animation
dbcol_detail = db.ani_detail

### ✏️ DataFrame 가져오기

In [187]:
import pandas as pd

# id, name, genres, content, tags, series_id 불러오기
df = pd.DataFrame(dbcol_detail.find({},{"id":1, "name":1, "genres":1, "content":1, "tags":1, "series_id":1}))
print(len(df))

6229


### ✏️ 결측값 확인 및 대체

In [188]:
def missing_values(data):
    missing = pd.DataFrame([
    df["content"].isnull().sum(), 
    df["genres"].isnull().sum(), 
    df["name"].isnull().sum(),
    df["tags"].isnull().sum(), 
    df["series_id"].isnull().sum()], index = ["content", "genres", "name", "tags", "series_id"])
    return missing


def replace_missing(data):
    data["genres"] = data["genres"].fillna("")
    data["content"] = data["content"].fillna("")
    data["name"] = data["name"].fillna("")
    data["tags"] = data["tags"].fillna("")
    data["series_id"] = data["series_id"].fillna("")
    

missing_values(df)
replace_missing(df)

missing_values(df)

,0
content,0
genres,0
name,0
tags,0
series_id,0


### ✏️ 불용어 설정

In [ ]:
# stopwords 다운로드
# import nltk
# nltk.download('stopwords')
# C:\Users\SSAFY\AppData\Roaming\nltk_data\corpora\stopwords에 korean 파일 생성 후 텍스트 추가하여 불러오기

from nltk.corpus import stopwords
stop_words = stopwords.words("korean")

### ✏️ 형태소 분석 후 DB 저장

In [ ]:
# db_ani = db.animation
dbcol_feat = db.ani_feature

In [ ]:
import re

# 한글, 숫자, 영문만 가져옴
def sub_special(s):
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)

In [ ]:
# Okt 활용
from konlpy.tag import Okt
okt = Okt()


# 리스트를 문자열로 변환
def list_to_str(list):
    str = " ".join(list)
    return str
    
# 형태소 분석
def morph_and_stopword(data):
    
    text = ""
    text += list_to_str(data["genres"]) + " "
    text += list_to_str(data["tags"]) + " "
    text += data["content"]
    
    # 특수문자 제거
    text = sub_special(text)
    
    #형태소 분석
    words = okt.morphs(text, stem=True)
    
    # 형태소 분석 결과 담을 리스트
    feature = []
    
    #불용어 처리
    for word in words:
        if word not in stop_words and len(word) > 1:
            feature.append(word)

    feat_str = list_to_str(feature)
    
    # 애니메이션 아이디, 분석 결과, 시리즈 아이디 DB 저장
    dbcol_feat.update_one({"id": data["id"]}, {"$set": {"id": data["id"], "series_id": data["series_id"], "feature": feature, "feat_str": feat_str}}, upsert = True)

In [ ]:
df.apply(morph_and_stopword, axis=1)
print("형태소 분석 및 DB 저장 완료")

### ✏️ TEST TF-IDF

In [189]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

df_feat = pd.DataFrame(dbcol_feat.find())

feat_str_list = df_feat["feat_str"].tolist()

# TF-IDF 분석
tf_idf = TfidfVectorizer()
tf_idf_matrix = tf_idf.fit_transform(feat_str_list)
print("TF-IDF 행렬의 크기(shape): ", tf_idf_matrix.shape)

cosine_sim = cosine_similarity(tf_idf_matrix, tf_idf_matrix)
print("코사인 유사도 연산 결과: ",cosine_sim.shape)

TF-IDF 행렬의 크기(shape):  (6229, 25185)
코사인 유사도 연산 결과:  (6229, 6229)


### ✏️ 애니메이션 아이디: 인덱스, 인덱스: 시리즈

In [190]:
id_to_idx = dict(zip(df["id"], df.index))
idx_to_series = dict(zip(df.index, df["series_id"]))
id_to_series = dict(zip(df["id"], df["series_id"]))

# float를 int로
for keys in idx_to_series:
    try:
        idx_to_series[keys] = int(idx_to_series[keys])
    except ValueError:
        pass

### ✏️ TEST RECOMM

In [191]:
# dict의 value로 key 찾기
def get_key(val, dict):
    for key, value in dict.items():
         if val == value:
                return key

In [196]:
def get_recommendations(id, cosine_sim=cosine_sim):
    
    # 선택한 애니메이션의 인덱스 가져옴
    idx = id_to_idx[id]
    
    # 해당 인덱스의 시리즈 아이디 가져옴
    series = idx_to_series[idx]
    
    # 해당 애니와의 유사도를 가져온다. (애니 - 전체 애니)
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도에 따라 애니 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 유사한 애니메이션 저장 리스트
    id_list = []
    
    # 유사한 애니메이션 중 같은 시리즈 체크 리스트
    series_list = [series]
    
    for index, value in enumerate(sim_scores):
        
        # 해당 애니의 index
        ani_idx =  get_key(value[0], id_to_idx)
        
        # 유사도가 0.95를 넘지 않고 이미 시리즈가 속해져있지 않다면 추가
        if value[1] < 0.95 and id_to_series[ani_idx] not in series_list:
            id_list.append(get_key(value[0], id_to_idx))
            
            # 다른 시리즈가 있는 경우에만 시리즈 체크 리스트에 추가
            if id_to_series[ani_idx] != "":
                series_list.append(id_to_series[ani_idx])
                
        # 10개가 채워지면 stop
        if len(id_list) == 10:
            break
            
    return id_list

### ✏️ 해당 애니메이션 아이디 값으로 결과 추출
| 애니메이션 번호 | 제목 |
| --- | --- |
| 40260 | 도쿄 리벤저스 part 1 |
| 16075 | 은혼 1기 |
| 38912 | 전생했더니 슬라임이었던 건에 대하여 1기 |
| 39431 | (무삭제) 귀멸의 칼날 |
| 40815 | (자막) 스파이 패밀리 |


In [197]:
relation_ani = get_recommendations(40260)
print(relation_ani)

[40161, 15650, 40181, 40674, 38912, 14374, 19080, 18925, 39022, 40677]


### ✨ 유사한 애니메이션 결과값 미리보기

In [201]:
def get_ani_detail(id):
    return dbcol_info.find_one({"id": id})

result = []

for r in relation_ani:
    result.append(get_ani_detail(r))

In [204]:
df_result = pd.DataFrame(result)
df_result[["id", "name", "genres", "tags", "series_id"]]

,id,name,genres,tags,series_id
0,40161,약캐 토모자키 군,"[로맨스, 드라마]","[성장, 게임, 아싸, 로맨스, 학교, 이 라이트노벨이 대단하다, 드라마, 소설 원작]",NaN
1,15650,흑장미 부인의 문방구,"[판타지, 미스터리]","[판타지, 미스터리]",NaN
2,40181,괴물사변,"[판타지, 액션, 미스터리]","[성장, 요괴 및 괴물, 열혈, 판타지, 액션, 미스터리]",NaN
3,40674,"굿바이, 돈 그리즈!",[드라마],"[우정, 발랄가볍, 드라마]",NaN
4,38912,전생했더니 슬라임이었던 건에 대하여 1기,"[이세계, 판타지, 액션, 개그, 모험]","[마법, 요괴 및 괴물, 전쟁, 이능력, 악마 또는 천사, 마왕, 환생, 발랄가볍,...",3856.0
5,14374,거신 고그,[미스터리],[미스터리],NaN
6,19080,텔레파시 소녀 란,[미스터리],[미스터리],NaN
7,18925,페르소나 트리니티 소울,"[판타지, 액션, 미스터리]","[성장, 이능력, 살인, 좀비, 무거움, 카리스마, 판타지, 액션, 미스터리, 학교]",3690.0
8,39022,물드는 세계의 내일로부터,"[판타지, 치유, 드라마]","[성장, 마법, 감동, 판타지, 치유, 학교, 드라마]",NaN
9,40677,극장판 DEEMO: 너의 연주는 마음을 수놓아,"[음악, 드라마, 판타지]","[감동, 판타지, 음악, 드라마]",NaN
